In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
# !wget https://mmaisharables.blob.core.windows.net/uc2/UC2_DATA.tar.gz

--2021-07-24 22:11:04--  https://mmaisharables.blob.core.windows.net/uc2/UC2_DATA.tar.gz
Resolving mmaisharables.blob.core.windows.net (mmaisharables.blob.core.windows.net)... 20.150.90.68
Connecting to mmaisharables.blob.core.windows.net (mmaisharables.blob.core.windows.net)|20.150.90.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9777118183 (9.1G) [application/gzip]
Saving to: ‘UC2_DATA.tar.gz’

 1% [                                       ] 142,614,528 8.51MB/s  eta 33m 35s^C


In [2]:
import torch

In [3]:
uc2 = torch.load("UC2_DATA/pretrain/uc2/ckpt/model_step_200000.pt")

In [4]:
for k, v in uc2.items():
    print(list(v.size()), '\t', k)

[250002, 768] 	 roberta.embeddings.word_embeddings.weight
[514, 768] 	 roberta.embeddings.position_embeddings.weight
[2, 768] 	 roberta.embeddings.new_token_type_embeddings.weight
[768] 	 roberta.embeddings.LayerNorm.weight
[768] 	 roberta.embeddings.LayerNorm.bias
[768, 2048] 	 roberta.img_embeddings.img_linear.weight
[768] 	 roberta.img_embeddings.img_linear.bias
[768] 	 roberta.img_embeddings.img_layer_norm.weight
[768] 	 roberta.img_embeddings.img_layer_norm.bias
[768] 	 roberta.img_embeddings.pos_layer_norm.weight
[768] 	 roberta.img_embeddings.pos_layer_norm.bias
[768, 7] 	 roberta.img_embeddings.pos_linear.weight
[768] 	 roberta.img_embeddings.pos_linear.bias
[2, 2048] 	 roberta.img_embeddings.mask_embedding.weight
[768] 	 roberta.img_embeddings.LayerNorm.weight
[768] 	 roberta.img_embeddings.LayerNorm.bias
[768, 768] 	 roberta.encoder.layer.0.attention.self.query.weight
[768] 	 roberta.encoder.layer.0.attention.self.query.bias
[768, 768] 	 roberta.encoder.layer.0.attention.self

In [38]:
import os
import sys
import argparse
sys.path.append("../")

import torch
from volta.config import BertConfig
from volta.encoders import BertForVLPreTraining


# # Inputs
# parser = argparse.ArgumentParser()
# parser.add_argument("--input_fn", type=str, default="Epoch20_LXRT.pth")
# parser.add_argument("--output_fn", type=str, default="lxmert_checkpoint_19.bin")
# parser.add_argument("--verbose", action="store_true", default=False)
# args = parser.parse_args()

# Load original checkpoint
# original_ckpt = torch.load(args.input_fn, map_location="cpu")

# Create corresponding VOLTA model
config_file = "../config/uc2_base.json"
config = BertConfig.from_json_file(config_file)
model = BertForVLPreTraining.from_pretrained("/science/image/nlp-datasets/emanuele/huggingface/xlm-roberta-base", config=config, default_gpu=True, from_hf=True)
trg_dict = model.state_dict()

# # Map original parameters onto VOLTA ones
# first_xlayer = config.tv_attn_sublayers[0]
# volta2original = dict()

model's visual targets are  ['0']


In [39]:
model

BertForVLPreTraining(
  (bert): BertModel(
    (embeddings): UC2Embeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (new_token_type_embeddings): Embedding(2, 768)
      (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (image_embeddings): Linear(in_features=2048, out_features=768, bias=True)
      (image_location_embeddings): Linear(in_features=7, out_features=768, bias=True)
      (image_token_type_embeddings): Embedding(2, 768)
      (image_layer_norm): FusedLayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
      (image_location_layer_norm): FusedLayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
      (v_LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
      (v_dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(


In [40]:
for k, v in trg_dict.items():
    print(list(v.size()), '\t', k)

[250002, 768] 	 bert.embeddings.word_embeddings.weight
[514, 768] 	 bert.embeddings.position_embeddings.weight
[2, 768] 	 bert.embeddings.new_token_type_embeddings.weight
[768] 	 bert.embeddings.LayerNorm.weight
[768] 	 bert.embeddings.LayerNorm.bias
[768, 2048] 	 bert.embeddings.image_embeddings.weight
[768] 	 bert.embeddings.image_embeddings.bias
[768, 7] 	 bert.embeddings.image_location_embeddings.weight
[768] 	 bert.embeddings.image_location_embeddings.bias
[2, 768] 	 bert.embeddings.image_token_type_embeddings.weight
[768] 	 bert.embeddings.image_layer_norm.weight
[768] 	 bert.embeddings.image_layer_norm.bias
[768] 	 bert.embeddings.image_location_layer_norm.weight
[768] 	 bert.embeddings.image_location_layer_norm.bias
[768] 	 bert.embeddings.v_LayerNorm.weight
[768] 	 bert.embeddings.v_LayerNorm.bias
[768, 768] 	 bert.encoder.layer.0.attention_self.query.weight
[768] 	 bert.encoder.layer.0.attention_self.query.bias
[768, 768] 	 bert.encoder.layer.0.attention_self.key.weight
[768]

In [41]:
volta2original = dict()
volta_keys = set(trg_dict.keys())

In [42]:
for k in uc2.keys():
    ln = str(k)
    ln = ln.replace("roberta", "bert")
    
    ln = ln.replace("img_embeddings", "embeddings")
    ln = ln.replace("img_linear", "image_embeddings")
    ln = ln.replace("pos_linear", "image_location_embeddings")
    ln = ln.replace("img_layer_norm", "image_layer_norm")
    ln = ln.replace("pos_layer_norm", "image_location_layer_norm")
    
    ln = ln.replace('attention.self', 'attention_self')
    ln = ln.replace('attention.output', 'attention_output')
    if '.layer.' in ln:
        num = int(ln.split(".")[3])
        new = 2*num + ('.intermediate.' in ln or '.output.' in ln)
        ln = ln.replace(f".{num}.", f".{new}.")
        
    ln = ln.replace("pooler", "t_pooler")
    ln = ln.replace("cls.dense", "cls.predictions.transform.dense")
    ln = ln.replace("cls.layer_norm", "cls.predictions.transform.LayerNorm")
    ln = ln.replace("cls.bias", "cls.predictions.bias")
    ln = ln.replace("cls.decoder", "cls.predictions.decoder")
    ln = ln.replace("itm_output", "cls.bi_seq_relationship")
    
    if ln not in volta_keys:
        print(k, "\t", ln)
    else:
        volta2original[ln] = k

roberta.img_embeddings.mask_embedding.weight 	 bert.embeddings.mask_embedding.weight
cls.decoder.bias 	 cls.predictions.decoder.bias
vis_cls.bias 	 vis_cls.predictions.bias
vis_cls.dense.weight 	 vis_cls.predictions.transform.dense.weight
vis_cls.dense.bias 	 vis_cls.predictions.transform.dense.bias
vis_cls.layer_norm.weight 	 vis_cls.predictions.transform.LayerNorm.weight
vis_cls.layer_norm.bias 	 vis_cls.predictions.transform.LayerNorm.bias
vis_cls.decoder.weight 	 vis_cls.predictions.decoder.weight
vis_cls.decoder.bias 	 vis_cls.predictions.decoder.bias
feat_regress.weight 	 feat_regress.weight
feat_regress.bias 	 feat_regress.bias
feat_regress.net.0.weight 	 feat_regress.net.0.weight
feat_regress.net.0.bias 	 feat_regress.net.0.bias
feat_regress.net.2.weight 	 feat_regress.net.2.weight
feat_regress.net.2.bias 	 feat_regress.net.2.bias
region_classifier.net.0.weight 	 region_classifier.net.0.weight
region_classifier.net.0.bias 	 region_classifier.net.0.bias
region_classifier.net.2.w

In [43]:
original_ckpt = uc2

In [44]:
# Apply mapping
for trg, src in volta2original.items():
    print(trg, '<-', src)
    assert trg_dict[trg].shape == original_ckpt[src].shape
    trg_dict[trg] = original_ckpt[src]
model.load_state_dict(trg_dict)

bert.embeddings.word_embeddings.weight <- roberta.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight <- roberta.embeddings.position_embeddings.weight
bert.embeddings.new_token_type_embeddings.weight <- roberta.embeddings.new_token_type_embeddings.weight
bert.embeddings.LayerNorm.weight <- roberta.img_embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias <- roberta.img_embeddings.LayerNorm.bias
bert.embeddings.image_embeddings.weight <- roberta.img_embeddings.img_linear.weight
bert.embeddings.image_embeddings.bias <- roberta.img_embeddings.img_linear.bias
bert.embeddings.image_layer_norm.weight <- roberta.img_embeddings.img_layer_norm.weight
bert.embeddings.image_layer_norm.bias <- roberta.img_embeddings.img_layer_norm.bias
bert.embeddings.image_location_layer_norm.weight <- roberta.img_embeddings.pos_layer_norm.weight
bert.embeddings.image_location_layer_norm.bias <- roberta.img_embeddings.pos_layer_norm.bias
bert.embeddings.image_location_embeddings.weig

<All keys matched successfully>

In [47]:
sum(model.state_dict()['bert.t_pooler.dense.bias'] == uc2['roberta.pooler.dense.bias'])

tensor(768)

In [50]:
sum(model.state_dict()['bert.encoder.layer.22.attention_self.key.bias'] == model.state_dict()['bert.encoder.layer.22.attention_self.v_key.bias'])

tensor(768)

In [53]:
(model.state_dict()['bert.encoder.layer.23.intermediate.dense.weight'] == model.state_dict()['bert.encoder.layer.23.intermediate.v_dense.weight']).sum()

tensor(2359296)

In [55]:
model.state_dict()['bert.encoder.layer.23.intermediate.dense.weight'].size(0) * model.state_dict()['bert.encoder.layer.23.intermediate.dense.weight'].size(1)

2359296

# UC2 data

In [2]:
import lmdb
import json

import msgpack
import msgpack_numpy
msgpack_numpy.patch()

db_name = "../../flickr30k/feat_th0.2_max100_min10"
nbb = "../../flickr30k/nbb_th0.2_max100_min10.json"
name2nbb = json.load(open(f'{nbb}'))

env = lmdb.open(f'{db_name}',readonly=True, create=False)
txn = env.begin(buffers=True)

In [3]:
filename = "flickr30k_002102835360.npz"
dump = txn.get(filename.encode('utf-8'))

In [4]:
dump

In [5]:
_nbb = name2nbb[filename]
_nbb

28

In [6]:
img_dump = msgpack.loads(dump, raw=False)

In [9]:
img_dump['norm_bb']

array([[0.      , 0.3228  , 0.209   , 0.7686  , 0.209   , 0.446   ],
       [0.2742  , 0.7817  , 0.4968  , 0.9985  , 0.2227  , 0.2164  ],
       [0.00809 , 0.2183  , 0.11865 , 0.2722  , 0.11053 , 0.05408 ],
       [0.      , 0.2064  , 0.1469  , 0.9985  , 0.1469  , 0.792   ],
       [0.2632  , 0.3118  , 0.5825  , 0.8413  , 0.319   , 0.53    ],
       [0.826   , 0.703   , 0.8657  , 0.801   , 0.03973 , 0.0979  ],
       [0.8726  , 0.8184  , 0.98    , 0.9985  , 0.1072  , 0.1797  ],
       [0.6143  , 0.4531  , 0.9526  , 0.957   , 0.3386  , 0.504   ],
       [0.564   , 0.2546  , 0.8647  , 0.7417  , 0.3008  , 0.4873  ],
       [0.      , 0.1522  , 0.774   , 0.848   , 0.774   , 0.6963  ],
       [0.769   , 0.7026  , 0.8555  , 0.8174  , 0.0863  , 0.1149  ],
       [0.2183  , 0.      , 0.6304  , 0.9985  , 0.4124  , 0.9985  ],
       [0.6836  , 0.1381  , 0.808   , 0.205   , 0.1245  , 0.0669  ],
       [0.73    , 0.      , 0.9995  , 0.9985  , 0.2693  , 0.9985  ],
       [0.657   , 0.05008 , 0.823 

In [10]:
0.4968-0.2742

0.22260000000000002

In [11]:
0.7686-0.3228

0.4458

In [40]:
our_ckpt = torch.load("/science/image/nlp-datasets/emanuele/checkpoints/iglue/conversions/pretrain/uc2_checkpoint_200000.bin")

In [41]:
our_ckpt.keys()

odict_keys(['bert.embeddings.word_embeddings.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.new_token_type_embeddings.weight', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.embeddings.image_embeddings.weight', 'bert.embeddings.image_embeddings.bias', 'bert.embeddings.image_location_embeddings.weight', 'bert.embeddings.image_location_embeddings.bias', 'bert.embeddings.image_token_type_embeddings.weight', 'bert.embeddings.image_layer_norm.weight', 'bert.embeddings.image_layer_norm.bias', 'bert.embeddings.image_location_layer_norm.weight', 'bert.embeddings.image_location_layer_norm.bias', 'bert.embeddings.v_LayerNorm.weight', 'bert.embeddings.v_LayerNorm.bias', 'bert.encoder.layer.0.attention_self.query.weight', 'bert.encoder.layer.0.attention_self.query.bias', 'bert.encoder.layer.0.attention_self.key.weight', 'bert.encoder.layer.0.attention_self.key.bias', 'bert.encoder.layer.0.attention_self.value.weight', 'bert.encoder.layer.0.atte

In [46]:
our_ckpt['cls.bi_seq_relationship.weight'].data

tensor([[-0.0035,  0.0005, -0.0106,  ...,  0.0034, -0.0266,  0.0057],
        [-0.0038,  0.0116, -0.0085,  ..., -0.0195,  0.0108,  0.0071]])

## Matching of models

In [36]:
import json
import argparse

with open(config_file) as data_file:
    arguments = json.load(data_file)
# arguments

parser = argparse.ArgumentParser(description="title", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
for key, val in arguments.items():
    setattr(parser, key, val)

In [4]:
uc2 = torch.load("UC2_DATA/pretrain/uc2/ckpt/model_step_200000.pt")

In [39]:
import os, sys
sys.path.append("UC2")
from model import VLXLMRForImageTextRetrieval

# opts = parser
# model = VLXLMRForImageTextRetrieval.from_pretrained(
#     opts.model_config, 
#     state_dict=uc2,
#     load_embedding_only=opts.load_embedding_only,
#     load_layer=opts.load_layer,
#     img_dim=IMG_DIM, 
#     margin=opts.margin
# )

FileNotFoundError: [Errno 2] No such file or directory: 'object_labels/img_label_objects.txt'